In [1]:
import os, glob
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os.path

In [2]:
#set paths
in_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/MICCAI_BraTS2020_TrainingData_unsorted'
t1_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t1/'
t2_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t2/'
t1ce_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t1ce/'
flair_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/flair/'
seg_path = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/seg/'
t1_path_test = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t1/'
t2_path_test = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t2/'
t1ce_path_test = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t1ce/'
flair_path_test = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/flair/'
seg_path_test = '/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/seg/'

# STEP 3 - Slicing and saving
SLICE_X = False
SLICE_Y = False
SLICE_Z = True
SLICE_DECIMATE_IDENTIFIER = 3

In [3]:
out_dir = [t1_path, t2_path, t1ce_path, flair_path, seg_path, t1_path_test, t2_path_test, t1ce_path_test, flair_path_test, seg_path_test]

for dirName in out_dir:
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")  

Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t1/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t2/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/t1ce/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/flair/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/train/seg/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t1/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t2/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t1ce/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/flair/  Created 
Directory  /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/seg/  Created 


In [4]:
#normalize the data
def normalize_data(img):
    return img / np.max(img)

# Read image or mask volume
def readImageVolume(imgPath, normalize=False):
    img = nib.load(imgPath).get_fdata()
    if normalize:
        return normalize_data(img)
    else:
        return img

#save slice to computer
def saveSlice(img, fname, path):
    img = np.uint8(img)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')

def saveSlice_seg(img, fname, path):
    img = np.uint8(img * 255)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')

# Slice image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    print(dimx, dimy, dimz)
    cnt = 0
    if file_path[-10:-7] != 'seg':
        if SLICE_X:
            cnt += dimx
            print('Slicing X: ')
            for i in range(dimx):
                saveSlice(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)
                
        if SLICE_Y:
            cnt += dimy
            print('Slicing Y: ')
            for i in range(dimy):
                saveSlice(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)
                
        if SLICE_Z:
            cnt += dimz
            print('Slicing Z: ')
            for i in range(dimz):
                saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)

    else:
        if SLICE_X:
            cnt += dimx
            print('Slicing X: ')
            for i in range(dimx):
                saveSlice_seg(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)
                
        if SLICE_Y:
            cnt += dimy
            print('Slicing Y: ')
            for i in range(dimy):
                saveSlice_seg(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)
                
        if SLICE_Z:
            cnt += dimz
            print('Slicing Z: ')
            for i in range(dimz):
                saveSlice_seg(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)
    return cnt

In [5]:
all_patients = os.listdir(in_path)

random.shuffle(all_patients)
train_size = int(0.8 * len(all_patients))
print(len(all_patients))
train_data = all_patients[:train_size]
test_data = all_patients[train_size:]

for i in train_data:
    if i != '.DS_Store' and os.path.isdir(os.path.join(in_path, i)):
        print(i)
        new_paths = os.path.join(in_path, i)
        new_paths_list = os.listdir(new_paths)
        for j in new_paths_list:
            file_path = os.path.join(new_paths, j)
            if file_path[-10:-7] == 'air':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, flair_path)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '_t1':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t1_path)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '_t2':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t2_path)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '1ce':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t1ce_path)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == 'seg':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, seg_path)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

for i in test_data:
    if i != '.DS_Store' and os.path.isdir(os.path.join(in_path, i)):
        print(i)
        new_paths = os.path.join(in_path, i)
        new_paths_list = os.listdir(new_paths)
        for j in new_paths_list:
            file_path = os.path.join(new_paths, j)
            if file_path[-10:-7] == 'air':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, flair_path_test)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '_t1':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t1_path_test)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '_t2':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t2_path_test)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == '1ce':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, t1ce_path_test)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

            if file_path[-10:-7] == 'seg':
                img = readImageVolume(file_path, False)
                print(file_path, img.shape, np.sum(img.shape), np.min(img), np.max(img))
                numOfSlices = sliceAndSaveVolumeImage(img, i, seg_path_test)
                print(f'\n{file_path}, {numOfSlices} slices created \n')

print('Slicing is done!')

r/Documents/JADS/YEAR_2/Thesis/data/test/seg/BraTS20_Training_330-slice154_z.png
/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/MICCAI_BraTS2020_TrainingData_unsorted/BraTS20_Training_330/BraTS20_Training_330_seg.nii.gz, 155 slices created 

/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/MICCAI_BraTS2020_TrainingData_unsorted/BraTS20_Training_330/BraTS20_Training_330_flair.nii.gz (240, 240, 155) 635 0.0 819.0
240 240 155
Slicing Z: 
[+] Slice saved: /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/flair/BraTS20_Training_330-slice154_z.png
/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/MICCAI_BraTS2020_TrainingData_unsorted/BraTS20_Training_330/BraTS20_Training_330_flair.nii.gz, 155 slices created 

/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/MICCAI_BraTS2020_TrainingData_unsorted/BraTS20_Training_330/BraTS20_Training_330_t1.nii.gz (240, 240, 155) 635 0.0 1453.0
240 240 155
Slicing Z: 
[+] Slice saved: /Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/test/t1/BraTS20_Training_330

In [18]:
nImg = normalize_data(img)
np.min(nImg), np.max(nImg), nImg.shape, type(nImg)

(0.0, 1.0, (240, 240, 155), numpy.ndarray)

In [25]:
import PIL
image = PIL.Image.open("/Users/jopbeuger/Documents/JADS/YEAR_2/Thesis/data/slices/test/BraTS20_Training_012-slice001_x.png")
width, height = image.size
print(width, height)

155 240
